In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
from peft import PeftModel
from tqdm import tqdm
import torch
import json
import math
import time
import os

MODEL_PATH='../../Llama-3.2-3B-Instruct'
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

QUESTIONS_PATH = "../rag_questions_json"

/home/ddp8196/miniconda3/envs/bdml/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model  = AutoModelForCausalLM.from_pretrained(MODEL_PATH, torch_dtype=torch.bfloat16)
model = PeftModel.from_pretrained(model, "./qlora_checkpoints/checkpoint-460").to(DEVICE).merge_and_unload().eval()
tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 27.37it/s]


In [3]:
def create_chat(question):
    return [
        {"role": "user", "content": f"{question}"},
    ]

### Example question

In [4]:
question = "What are the main indicators that were chosen to study in order to understand and forecast the evolution of carbon emissions on a country-scale, and why were they chosen?"
prompt_with_context = tokenizer.apply_chat_template(
    create_chat(question), 
    tokenize=False, 
    add_generation_prompt=True
)
print(prompt_with_context)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 21 Apr 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

What are the main indicators that were chosen to study in order to understand and forecast the evolution of carbon emissions on a country-scale, and why were they chosen?<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [5]:
streamer = TextStreamer(tokenizer, skip_prompt=True)

input_ids = tokenizer.encode(prompt_with_context, return_tensors="pt").to(DEVICE)
outputs = model.generate(
    input_ids,
    max_new_tokens=500,
    pad_token_id=128004,
    eos_token_id=128009,
    streamer=streamer,
    do_sample=True,
)

To study and forecast the evolution of carbon emissions on a country-scale, researchers and policymakers typically use a combination of indicators that capture various aspects of the energy sector, economy, and human behavior. Some of the main indicators used to study and forecast carbon emissions include:

1. **Energy intensity**: This refers to the amount of energy consumed per unit of GDP (Gross Domestic Product) or per capita. Energy intensity is a key driver of greenhouse gas emissions, as it reflects the amount of energy required to produce goods and services.
2. **Economic growth**: Economic growth is a key driver of energy demand and carbon emissions. As a country's economy grows, so does its energy demand, which can lead to increased emissions.
3. **Renewable energy share**: The share of renewable energy sources in the energy mix is a key indicator of a country's transition towards a low-carbon economy. An increasing share of renewables can reduce emissions, while a decreasing

### Evaluation (Perplexity)

In [6]:
def create_target_chat(question, answer):
    return [
        {"role": "user", "content": f"{question}"},
        {"role": "assistant", "content": f"{answer}"},
    ]

In [7]:
loss = 0
losses = list()
question_filenames = os.listdir(QUESTIONS_PATH)

for question_filename in tqdm(question_filenames):
    with open(os.path.join(QUESTIONS_PATH, question_filename), 'r', encoding='utf-8') as f:
        qa = json.load(f)

    target_chat = tokenizer.apply_chat_template(
        create_target_chat(qa["question"], qa["answer"]), 
        tokenize=False, 
        add_generation_prompt=False
    )
    tokens = tokenizer(target_chat, return_tensors="pt", truncation=False, padding=False, add_special_tokens=False).to(DEVICE)
    tokens = {k: v.to(DEVICE) for k, v in tokens.items()}

    prompt = tokenizer.apply_chat_template(
        create_chat(qa["question"]), 
        tokenize=False, 
        add_generation_prompt=True
    )
    ignore_idx = tokenizer.encode(prompt, return_tensors="pt", truncation=False, padding=False, add_special_tokens=False).shape[1]

    filtered_labels = tokens["input_ids"].clone()
    ignore_mask = torch.zeros_like(filtered_labels, dtype=torch.bool)
    ignore_mask[0, :ignore_idx] = True
    filtered_labels[ignore_mask] = -100

    with torch.no_grad():
        outputs = model(**tokens, labels=filtered_labels)
        losses.append(outputs.loss.item())
        loss += outputs.loss.item()

perplexity = math.exp(loss/len(question_filenames))
print(f"Perplexity: {perplexity:.2f}")

100%|██████████| 747/747 [00:27<00:00, 27.59it/s]

Perplexity: 10.17


### Evaluation (Average time per request)

In [8]:
begin = time.time()

for question_filename in tqdm(question_filenames):
    with open(os.path.join(QUESTIONS_PATH, question_filename), 'r', encoding='utf-8') as f:
        qa = json.load(f)

    prompt = tokenizer.apply_chat_template(
        create_chat(qa["question"]), 
        tokenize=False, 
        add_generation_prompt=True
    )
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(DEVICE)
    model.generate(
        input_ids,
        max_new_tokens=500,
        pad_token_id=128004,
        eos_token_id=128009,
        do_sample=False,
        top_p=1.0,
    )

print(f"Average time per request: {(time.time() - begin) / len(question_filenames):.2f} seconds")

  0%|          | 0/747 [00:00<?, ?it/s]/home/ddp8196/miniconda3/envs/bdml/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
100%|██████████| 747/747 [2:51:33<00:00, 13.78s/it]  

Average time per request: 13.78 seconds
